In [ ]:
    import warnings
    warnings.filterwarnings("ignore")

    import pandas as pd
    import numpy as np
    import sys, os
    import pickle

    from sklearn.model_selection import train_test_split
    from class_SurvivalQuilts import SurvivalQuilts
    from utils_eval import calc_metrics


    #=================================================================#
    ##### USER-DEFINED FUNCTIONS
    def f_get_Normalization(X, norm_mode):
    """
    Normalize the features based on the selected normalization mode.

    Parameters:
    X: Feature matrix (numpy array or pandas DataFrame).
    norm_mode: Normalization mode ('standard' for Z-score normalization, 'normal' for Min-Max scaling).

    Returns:
    Normalized feature matrix.
    """
    X = np.array(X) # Convert to numpy array if not already
    num_Patient, num_Feature = np.shape(X)

    if norm_mode == 'standard': # Zero mean, unit variance normalization
    for j in range(num_Feature):
    if np.std(X[:, j]) != 0:
    X[:, j] = (X[:, j] - np.mean(X[:, j])) / np.std(X[:, j])
    else:
    X[:, j] = (X[:, j] - np.mean(X[:, j]))
    elif norm_mode == 'normal': # Min-Max normalization
    for j in range(num_Feature):
    X[:, j] = (X[:, j] - np.min(X[:, j])) / (np.max(X[:, j]) - np.min(X[:, j]))
    else:
    print("INPUT MODE ERROR!")

    return X


    #=================================================================#


    ##### DATASET SELECTION
    SEED = 1111
    data_mode = 'metabric' # {'metabric', 'support'}

    if data_mode == 'metabric':
    # Load feature data and labels
    X = pd.read_csv('./sample data/metabric_cleaned_features_final.csv')
    tmp = pd.read_csv('./sample data/metabric_label.csv')
    T = tmp[['event_time']]
    Y = tmp[['label']]
    time_interval_ = 10.0
    SEED = 4321 # Seed for random state

    # Create a results folder if it doesn't exist
    tmp_folder = './results/' + data_mode + ' (seed ' + str(SEED) + ')/'
    if not os.path.exists(tmp_folder):
    os.makedirs(tmp_folder)

    # Define evaluation time horizons based on quantiles of event times
    eval_time_horizons = [
    int(T[Y.iloc[:, 0] == 1].quantile(0.25)),
    int(T[Y.iloc[:, 0] == 1].quantile(0.50)),
    int(T[Y.iloc[:, 0] == 1].quantile(0.75))
    ]

    # Split the dataset into training and test sets
    tr_X, te_X, tr_T, te_T, tr_Y, te_Y = train_test_split(X, T, Y, test_size=0.2, random_state=1234)

    # Apply normalization to the feature matrices
    tr_X = f_get_Normalization(tr_X, norm_mode='standard') # You can change to 'normal' if needed
    te_X = f_get_Normalization(te_X, norm_mode='standard')

    # Initialize and train the SurvivalQuilts model
    model_sq = SurvivalQuilts()
    model_sq.train(tr_X, tr_T, tr_Y)

    # Save the trained model
    filename = './results/' + 'SurvivalQuilts.sav'
    pickle.dump(model_sq, open(filename, 'wb'))

    # Make predictions for the test set
    pred = model_sq.predict(te_X, eval_time_horizons)

    # Evaluate the model performance at each time horizon using calc_metrics
    for e_idx, eval_time in enumerate(eval_time_horizons):
    c_index, brier_score = calc_metrics(tr_T, tr_Y, te_T, te_Y, pred[:, e_idx], eval_time)
    print(f'Evaluation at time {eval_time}: C-index = {c_index:.4f}, Brier Score = {brier_score:.4f}')

